In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm_notebook
np.set_printoptions(suppress=True)
from sklearn import preprocessing
from tqdm import tqdm
import decimal
from itertools import combinations
from random import shuffle
from time import sleep
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import gc
import datetime
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import os
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
import random
import lightgbm as lgb

In [2]:
#Always seed the randomness of this universe
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(seed=51)

In [4]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [5]:
%%time
train = pd.read_csv('../input/train6.csv')
print("train finished")

train finished
CPU times: user 1min 33s, sys: 6.05 s, total: 1min 39s
Wall time: 1min 40s


In [6]:
%%time
test = pd.read_csv('../input/test6.csv')
print("test finished")

test finished
CPU times: user 1min 15s, sys: 4.12 s, total: 1min 19s
Wall time: 1min 20s


In [7]:
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(train.shape,test.shape,sample_submission.shape)

(590540, 1209) (506691, 1208) (506691, 2)


In [8]:
#y = train['isFraud']
#del train['isFraud']
gc.collect()
print(train.shape,test.shape)

(590540, 1209) (506691, 1208)


In [15]:
dropped_list = list(train.columns[-129:])
train.drop(dropped_list,inplace=True,axis=1)
test.drop(dropped_list,inplace=True,axis=1)
gc.collect()

2652

In [ ]:
"""
importances = pd.read_csv('./importances/importances1.csv')
print(importances.shape)
importances = importances[:700]
features = [x for x in list(importances['index']) if x not in ['TransactionID','TransactionDT']]
train = train[features]
test = test[features]
"""

In [16]:
gc.collect()

0

In [18]:
print(train.shape,test.shape)

(590540, 1080) (506691, 1079)


In [19]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_17,id_18,id_19,id_20,id_30,id_31,id_32,id_33,id_35,id_36,id_37,id_38,DeviceInfo,dayofyear,hour,dayofweek,day,card1_999999,card1_fraudulent,card2_999999,card2_fraudulent,card3_999999,card3_185.0,card4_discover,card5_nan,card5_999999,card5_137.0,card5_118.0,card5_100.0,card5_150.0,card5_229.0,card5_117.0,card5_166.0,card5_126.0,is_debit,ProductCD_C,ProductCD_W,addr1_999999,addr1_nan,addr1_888888.0,addr1_191.0,addr1_184.0,addr1_436.0,addr1_143.0,addr1_205.0,addr1_194.0,addr1_494.0,addr1_444.0,addr2_999999,addr2_nan,addr2_96.0,addr2_999999.0,TransactionAmt_decimal,TransactionAmt_check,dist1_999999,dist1_nan,dist1_91.0,dist1_130.0,dist1_116.0,dist1_108.0,dist1_86.0,dist1_95.0,dist1_114.0,dist1_33.0,dist1_12.0,dist1_27.0,dist1_50.0,dist1_55.0,dist1_54.0,dist1_42.0,dist1_47.0,dist1_35.0,dist1_43.0,dist1_49.0,dist2_999999,dist2_nan,dist2_0.0,dist2_9.0,dist2_1.0,dist2_10.0,dist2_68.0,dist2_999999.0,dist2_7.0,P_emaildomain_prefix,P_emaildomain_suffix,P_emaildomain_lesscategory,P_emaildomain_999999,P_emaildomain_nan,P_emaildomain_mail.com,P_emaildomain_outlook.es,P_emaildomain_aim.com,P_emaildomain_outlook.com,P_emaildomain_roadrunner.com,P_emaildomain_verizon.net,P_emaildomain_att.net,P_emaildomain_sbcglobal.net,P_emaildomain_rocketmail.com,P_emaildomain_windstream.net,P_emaildomain_hotmail.fr,P_emaildomain_protonmail,...,group_10_median,group_10_min,group_10_max,group_10_sum,group_10_std,group_10_kurtosis,group_11_mean,group_11_median,group_11_min,group_11_max,group_11_sum,group_11_std,group_11_kurtosis,group_12_mean,group_12_median,group_12_min,group_12_max,group_12_sum,group_12_std,group_12_kurtosis,group_13_mean,group_13_median,group_13_min,group_13_max,group_13_sum,group_13_std,group_13_kurtosis,group_14_mean,group_14_median,group_14_min,group_14_max,group_14_sum,group_14_std,group_14_kurtosis,group_15_mean,group_15_median,group_15_min,group_15_max,group_15_sum,group_15_std,group_15_kurtosis,id_01is_null,id_02is_null,id_03is_null,id_04is_null,id_05is_null,id_06is_null,id_07is_null,id_08is_null,id_09is_null,id_10is_null,id_11is_null,id_03multiplyid_04

In [20]:
cat_cols = ['ProductCD','card1','card2','card3','card4','card5','dist1','dist2','P_emaildomain','R_emaildomain','DeviceInfo']
print(len(cat_cols))

11


In [21]:
train[cat_cols].nunique()

ProductCD           5
card1            3214
card2             298
card3              10
card4               5
card5              19
dist1             499
dist2              16
P_emaildomain      60
R_emaildomain      61
DeviceInfo        187
dtype: int64

In [22]:
for tuple_ in tqdm(list(combinations(cat_cols,3))):
    f1 = tuple_[0]
    f2 = tuple_[1]
    f3 = tuple_[2]
    column_name = f1 + '_' + f2 + '_' + f3
    train[column_name + '_isnull'] = train[column_name].isnull()*1
    train[column_name + '_isnull'] = train[column_name + '_isnull'].astype(int)
    test[column_name + '_isnull'] = test[column_name].isnull()*1
    test[column_name + '_isnull'] = test[column_name + '_isnull'].astype(int)

100%|██████████| 165/165 [00:10<00:00, 26.26it/s]


In [23]:
for tuple_ in tqdm(list(combinations(cat_cols,3))):
    f1 = tuple_[0]
    f2 = tuple_[1]
    f3 = tuple_[2]
    column_name = f1 + '_' + f2 + '_' + f3
    train[column_name] = train[column_name].fillna('999999')
    test[column_name] = test[column_name].fillna('999999')
    le = LabelEncoder()
    le.fit(list(train[column_name]) + list(test[column_name]))
    train[column_name] = le.transform(train[column_name])
    test[column_name] = le.transform(test[column_name])
    del le,column_name,f1,f2,f3
print(train.shape,test.shape)

100%|██████████| 165/165 [03:04<00:00,  1.66it/s]

(590540, 1245) (506691, 1244)


In [24]:
print(train.shape,test.shape)

(590540, 1245) (506691, 1244)


# Training

In [25]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [26]:
%%time
train = reduce_mem_usage(train)

Mem. usage decreased to 1174.24 Mb (79.1% reduction)
CPU times: user 3min 9s, sys: 2min 27s, total: 5min 36s
Wall time: 5min 36s


In [33]:
train.dtypes.to_csv('../input/train6_dtypes.csv',index=True)
test.dtypes.to_csv('../input/test6_dtypes.csv',index=True)

In [27]:
%%time
test = reduce_mem_usage(test)

Mem. usage decreased to 1025.39 Mb (78.7% reduction)
CPU times: user 2min 23s, sys: 1min 26s, total: 3min 49s
Wall time: 3min 49s


In [28]:
gc.collect()

0

In [34]:
%%time
train.to_csv('../input/train6.csv',index=False)

CPU times: user 11min 51s, sys: 2.66 s, total: 11min 54s
Wall time: 11min 53s


In [35]:
%%time
test.to_csv('../input/test6.csv',index=False)

CPU times: user 10min 32s, sys: 2.14 s, total: 10min 34s
Wall time: 10min 34s
